In [1]:
!pip install confluent_kafka
!pip install elasticsearch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 53.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.3/914.3 kB 19.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 3.3 MB/s eta 0:00:00


In [2]:
from confluent_kafka import Consumer
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, DoubleType
from pyspark.ml import PipelineModel
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from keras.models import load_model
import numpy as np
import json
from pymongo import MongoClient
import logging
import os
from keras.losses import MeanSquaredError
import time
import pickle
from confluent_kafka import Consumer, KafkaException
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from tensorflow.keras.models import load_model
from elasticsearch import Elasticsearch

2025-07-13 14:28:38.977160: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752416919.230043      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752416919.303553      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# ===== Configuration =====
# Set logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

KAFKA_CONFIG = {
    'bootstrap.servers': 'pkc-312o0.ap-southeast-1.aws.confluent.cloud:9092',
    'security.protocol': 'SASL_SSL',
    'sasl.mechanisms': 'PLAIN',
    'sasl.username': 'BECRWYYPQA4FNGPW',
    'sasl.password': 'FFQ544dYKqJZqMklOrd4V8ApHKLTQhCxm3522zoegl9ngAF3IubQpJ0rlj15D2zF',
    'group.id': 'fraud_detector',
    'auto.offset.reset': 'earliest',
    'enable.auto.commit': False
}

TOPIC = "fraud_1"

In [4]:
# ============ Load Model & Scaler ============
logger.info("Loading model and scaler...")
autoencoder = load_model("/kaggle/input/dataset2/autoencoder_fraud_model.h5", compile=False)

with open("/kaggle/input/dataset2/minmax_scaler.pkl", "rb") as f:
    scaler = pickle.load(f)

features = [
    "amount", "current_age", "per_capita_income", "yearly_income",
    "total_debt", "Debt_to_Income_Ratio", "credit_limit", "credit_score"
]

mse_threshold = 0.99

2025-07-13 14:28:57.733565: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [5]:
consumer = Consumer(KAFKA_CONFIG)
consumer.subscribe([TOPIC])
print(f"Listening to topic: {TOPIC}")

Listening to topic: fraud_1


In [ ]:
try:
    while True:
        msg = consumer.poll(1.0)
        if msg is None:
            continue
        if msg.error():
            raise KafkaException(msg.error())

        try:
            record = json.loads(msg.value().decode("utf-8"))
            df_input = pd.DataFrame([record])[features]

            # Chuẩn hoá với MinMaxScaler
            scaled_input = scaler.transform(df_input)

            # Dự đoán bằng AutoEncoder
            reconstructed = autoencoder.predict(scaled_input)
            mse = np.mean(np.square(scaled_input - reconstructed), axis=1)[0]
            is_fraud = int(mse > mse_threshold)

            # Hiển thị kết quả
            print("\nNew Transaction Received:")
            print(f"Transaction ID: {record.get('transaction_id', 'N/A')}")
            print(f"Reconstruction MSE: {mse:.6f}")
            print(f"Fraud Detected: {'YES' if is_fraud else 'NO'}")

        except Exception as e:
            print(f"Error processing message: {e}")
            continue

except KeyboardInterrupt:
    print("Stopping consumer...")

finally:
    consumer.close()

%6|1752416940.391|GETSUBSCRIPTIONS|rdkafka#consumer-1| [thrd:main]: Telemetry client instance id changed from AAAAAAAAAAAAAAAAAAAAAA to wiTBM6SQSyyMHsLrM8L5Iw


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step

New Transaction Received:
Transaction ID: 7479444
Reconstruction MSE: 0.000027
Fraud Detected: NO
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step

New Transaction Received:
Transaction ID: 7486725
Reconstruction MSE: 0.000235
Fraud Detected: NO
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step

New Transaction Received:
Transaction ID: 7506488
Reconstruction MSE: 0.000077
Fraud Detected: NO
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step

New Transaction Received:
Transaction ID: 7522354
Reconstruction MSE: 0.000025
Fraud Detected: NO
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step

New Transaction Received:
Transaction ID: 7525132
Reconstruction MSE: 0.000164
Fraud Detected: NO
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step

New Transaction Received:
Transaction ID: 7533803
Reconstruction MSE: 0.003304
Fraud Detected: NO
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step

New Transaction Received:
Transaction ID: 7541607
Reconstruction MSE: 0.000371
Fraud Detected: NO
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step

N